# Sensitivity to Budget

In [1]:
import numpy as np
import time
import pickle
import sys
import os
import csv

from RhoAndBeta import CalcRhoAndBetaVectors
from UtilitiesOptimization import SubgrAlgSavPrimDualObjInd, \
    SubgrAlgSavPrimDualObjFn_L2Ind
from SimulationCode import ExperIndL2IndAndGreedy, Exper_Ind_L2_L2Ind_Greedy

In [2]:
## Read the data created in CreateDataJupNot
suffix='InstanceInfo/Ins1LongRun/'
num_impressions=pickle.load(open(suffix+'num_impressions'+'.p',"rb"))
numCampaigns=pickle.load(open(suffix+'numCampaigns'+'.p',"rb"))
num_edges=pickle.load(open(suffix+'num_edges'+'.p',"rb"))
index_Imps=pickle.load(open(suffix+'index_Imps'+'.p',"rb"))
index_sizeCamps=pickle.load(open(suffix+'index_sizeCamps'+'.p',"rb"))
index_startCamp=pickle.load(open(suffix+'index_startCamp'+'.p',"rb"))
vector_m=pickle.load(open(suffix+'vector_m'+'.p',"rb"))
vector_maxbid=pickle.load(open(suffix+'vector_maxbid'+'.p',"rb"))
vector_r=pickle.load(open(suffix+'vector_r'+'.p',"rb"))
vector_s=(pickle.load(open(suffix+'vector_s'+'.p',"rb"))).astype(int)
ext_s=pickle.load(open(suffix+'ext_s'+'.p',"rb"))
adverPerImp=(pickle.load(open(suffix+'adverPerImp'+'.p',"rb"))).astype(int)
UB_bidsPerImp = pickle.load(open(suffix+'UB_bids'+'.p',"rb"))
vector_ctr=pickle.load(open(suffix+'vector_ctr'+'.p',"rb"))
vector_rctr=pickle.load(open(suffix+'vector_rctr'+'.p',"rb"))
probImp=pickle.load(open(suffix+'probImp'+'.p',"rb"))

In [3]:
## If this parameter is true then first price auctions (\beta_i(b) =b) are used, otherwise second price.
firstPrice = True
## If this parameter is true a parameterSearch will be performd and .csv will be made,
parameterSearch = False
## We use $\tau_k = 1/m_k$
expForTau=-1.0
tau = np.power(vector_m,expForTau)
UB_bids = UB_bidsPerImp[index_Imps]

suffix='ResultsSensitivity/'
current_directory = os.getcwd()
results_directory = os.path.join(current_directory, suffix)
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Parameter Search

This parameter search looks for a constant $C$, such that the dual method shows 
empirical convergence. Given that here we only look for training convergence, we don't 
need to cross-validate or similar.

In [4]:
if parameterSearch:
    input_var = input("This will execute a simple parameter search.\
    If you just wanted to run just a full long run do parameterSearch= False  \
    and kill this execution. To continue press enter.")
    c=np.array([0.5,0.1,0.05,0.01,0.005,0.001,0.0005,0.0001,0.00005,0.00001])
    p_grad_Type = 0
    num_it = 1000
    it_per_cal = 250
    init_lam=np.zeros((numCampaigns))
    alphas_pure=np.fromfunction(lambda i, j: (1/(np.sqrt(i + 1))), \
        (num_it, 1), dtype=int)[:,0] 
    
#     dualObjFn, primalObjFn, dualObjFnAvg, primalObjFnAvg, budget_used, \
#     budget_LamAvgUse, dual_vars, dual_varsAvg = [], [], [], [], [], [], [], []

    nameResults='ParameterSearch'+'It_'+str(num_it)
    f = open(suffix+nameResults+'.csv', 'wt')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow( ('Auction Type', 'Problem Type', 'Ite', 'Cte', 'DualFnValue',\
        'PrimalValue','DualFnValueAvg','PrimalFnValueAvg') )
    
    print('Using First Price Auctions')
    firstPrice = True
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))

            
    print('Using Second Price Auctions')
    firstPrice = False
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
    f.close()

This will execute a simple parameter search.    If you just wanted to run just a full long run do parameterSearch= False      and kill this execution. To continue press enter.
Using First Price Auctions
c_val:  0.5, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  0.1, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  0.05, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  0.01, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  0.005, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  0.001, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  0.0005, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  0.0001, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with indicator
c_val:  5e-05, Methods: Indicator Case ,L2 penalization wout indicator ,L2 with i

# The following value are obtained just by Looking at The Parameter Search .csv

In [5]:
## Best constants
consBestFP = [0.0001, 0.5, 0.5]
consBestSP = [0.0001, 0.5, 0.5]

# Run Sensitivity to Budgets

In [7]:
np.random.seed(1234)
seq_vectorM = []
for i in range(1,31):
    seq_vectorM.append(np.ones(numCampaigns)*i*10)

num_itInd, num_itL2, num_itL2Ind = 10000, 10000, 10000
init_lam = np.zeros(numCampaigns)

shuffle = False
sim = 100

np.random.seed(12345)

In [8]:
## This is to indicate the type of utility function and subgradient associated to it.
p_grad_TypeInd = 0
p_grad_TypeL2 = 1
p_grad_TypeL2Ind = 2

## If shuffle = True we use exactly \sum_i s_i impression arrivals but the type is choosen uniformly
## at random. If false, also the impression 



firstPrice = True
print('Running Experiments Assuming First Price')
alphasInd = consBestFP[0] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itInd)])
alphasL2 = consBestFP[1] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itL2)])
alphasL2Ind = consBestFP[2] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itL2Ind)])
[dictToRetInd_FP, dictToRetL2_FP, dictToRetL2Ind_FP, dictToRetGr_FP] = \
    Exper_Ind_L2_L2Ind_Greedy(numCampaigns, num_impressions, num_edges, index_Imps, \
    index_sizeCamps, vector_r, vector_s, vector_ctr, alphasInd, num_itInd, alphasL2, num_itL2,\
    alphasL2Ind, num_itL2Ind, p_grad_TypeInd, p_grad_TypeL2, p_grad_TypeL2Ind, init_lam, \
    seq_vectorM, UB_bidsPerImp, adverPerImp, firstPrice, sim = sim, shuffle = shuffle)


Running Experiments Assuming First Price
Budget per Campaign: 10.0
Running Two-Phase: Indicator, L2, L2 + Indicator
Running the Primal-Dual Algorithms took: 234.53242301940918
Running the Whole Simulation Code took: 772.5199501514435secs.
Budget per Campaign: 20.0
Running Two-Phase: Indicator, L2, L2 + Indicator
Running the Primal-Dual Algorithms took: 230.8297519683838
Running the Whole Simulation Code took: 909.379851102829secs.
Budget per Campaign: 30.0
Running Two-Phase: Indicator, L2, L2 + Indicator
Running the Primal-Dual Algorithms took: 232.66705775260925
Running the Whole Simulation Code took: 978.5587019920349secs.
Budget per Campaign: 40.0
Running Two-Phase: Indicator, L2, L2 + Indicator
Running the Primal-Dual Algorithms took: 233.60172176361084
Running the Whole Simulation Code took: 1052.6240220069885secs.
Budget per Campaign: 50.0
Running Two-Phase: Indicator, L2, L2 + Indicator
Running the Primal-Dual Algorithms took: 239.20283222198486
Running the Whole Simulation Code

In [10]:
## Writting to file
from Utilities import CreateTableSensitivity


TableSensFP = CreateTableSensitivity(dictToRetInd_FP, dictToRetL2_FP, dictToRetL2Ind_FP,\
                                     dictToRetGr_FP, seq_vectorM, sim)

nameToSaveFP = 'TableSensFP.csv'

with open(suffix+nameToSaveFP, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Budget', 'sim',\
        'Ind-Profit', 'Ind-Revenue', 'Ind-Cost', 'Ind-BidsMade',\
        'Ind-BidsWon', 'Ind-ClicksWon', 'Ind-%BudgetUsed',\
        'L2-Profit', 'L2-Revenue', 'L2-Cost', 'L2-BidsMade',\
        'L2-BidsWon', 'L2-ClicksWon', 'L2-%BudgetUsed',\
        'L2Ind-Profit', 'L2Ind-Revenue', 'L2Ind-Cost', 'L2Ind-BidsMade',\
        'L2Ind-BidsWon', 'L2Ind-ClicksWon', 'L2Ind-%BudgetUsed',\
        'Gr-Profit', 'Gr-Revenue', 'Gr-Cost', 'Gr-BidsMade',\
        'Gr-BidsWon', 'Gr-ClicksWon', 'Gr-%BudgetUsed'])
    [writer.writerow(r) for r in TableSensFP]

In [ ]:
# def CreateNiceDictionariesForImpAndCampPos(numCampaigns, num_impressions, num_edges, \
#     index_Imps, index_sizeCamps):
#     dictImpPerCamp = {}
#     dictCampPerImp = {}
#     count = 0
#     for campNumber, impsInCamp in enumerate(index_sizeCamps):
#         dictImpPerCamp[campNumber] = index_Imps[count:(count+impsInCamp)]
#         count += impsInCamp

#     ## This is slow, but is sufficient

#     for impType in range(num_impressions):
#         dictCampPerImp[impType] = []

#     for camp in range(numCampaigns):
#         for impType in dictImpPerCamp[camp]:
#             dictCampPerImp[impType].append(camp)

#     return dictImpPerCamp, dictCampPerImp

# def CreateMatR_ctr_Rctr_AndRankings(dictImpPerCamp, dictCampPerImp, vector_r, vector_ctr,\
#         vector_qctr, index_Imps, num_edges, num_impressions, UB_bids, adverPerImp):
#     mat_r_by_Imp = np.zeros((numCampaigns, num_impressions))
#     mat_ctr = np.zeros((numCampaigns, num_impressions))
#     mat_rctr_by_Imp = np.zeros((numCampaigns, num_impressions))
#     ranking = {}
#     optBidGreedySP = {}
#     optBidGreedyFP = {}
#     for impType in range(num_impressions):
#         indexes = np.arange(num_edges)[(index_Imps == impType)]
# #         print('dictCampPerImp[impType]: '+str(dictCampPerImp[impType]))
# #         print('indexes: '+str(indexes))
#         mat_r_by_Imp[dictCampPerImp[impType], impType] =  vector_r[indexes]
#         mat_ctr[dictCampPerImp[impType], impType] =  vector_ctr[indexes]
#         mat_rctr_by_Imp[dictCampPerImp[impType], impType] =  vector_qctr[indexes]
#         ranking[impType] = (np.argsort(-vector_qctr[indexes])).astype(int)
#         posRankingGlobal = indexes[ranking[impType]]
#         optBidGreedySP[impType] = np.minimum(vector_qctr[posRankingGlobal], UB_bids[posRankingGlobal])
#         optBidGreedyFP[impType] =\
#             np.minimum(vector_qctr[posRankingGlobal]*adverPerImp[i]/(adverPerImp[i]+1.0), UB_bids[posRankingGlobal])
#     return [mat_r_by_Imp, mat_ctr, mat_rctr_by_Imp, ranking, optBidGreedySP,\
#         optBidGreedyFP]
        
        
        
    
    

In [ ]:
# def CreateMatR_ctr_Rctr_AndRankings(numCampaigns, num_impressions, num_edges, \
#             index_Imps, index_sizeCamps, vector_r, vector_ctr, \
#             vector_qctr,  UB_bid, adverPerImp):
#     _, dictCampPerImp = CreateNiceDictionariesForImpAndCampPos(numCampaigns,\
#         num_impressions, num_edges, index_Imps, index_sizeCamps)

#     mat_r_by_Imp = np.zeros((numCampaigns, num_impressions))
#     mat_ctr = np.zeros((numCampaigns, num_impressions))
#     mat_rctr_by_Imp = np.zeros((numCampaigns, num_impressions))
#     ranking = {}
#     optBidGreedySP = {}
#     optBidGreedyFP = {}
#     UB_bids = UB_bid[index_Imps]
#     for impType in range(num_impressions):
#         indexes = np.arange(num_edges)[(index_Imps == impType)]
# #         print('dictCampPerImp[impType]: '+str(dictCampPerImp[impType]))
# #         print('indexes: '+str(indexes))
#         mat_r_by_Imp[dictCampPerImp[impType], impType] =  vector_r[indexes]
#         mat_ctr[dictCampPerImp[impType], impType] =  vector_ctr[indexes]
#         mat_rctr_by_Imp[dictCampPerImp[impType], impType] =  vector_qctr[indexes]
#         rankingWrtIndexes = list((np.argsort(-vector_qctr[indexes])).astype(int))
#         ranking[impType] = [dictCampPerImp[impType][z] for z in rankingWrtIndexes]
#         posRankingGlobal = [indexes[z] for z in rankingWrtIndexes]
#         optBidGreedySP[impType] = np.minimum(vector_qctr[posRankingGlobal], UB_bids[posRankingGlobal])
#         optBidGreedyFP[impType] =\
#             np.minimum(vector_qctr[posRankingGlobal]*adverPerImp[impType]/(adverPerImp[impType]+1.0), UB_bids[posRankingGlobal])
#     return [mat_r_by_Imp, mat_ctr, mat_rctr_by_Imp, ranking, optBidGreedySP,\
#         optBidGreedyFP]

# def CreateNiceDictionariesForImpAndCampPos(numCampaigns, num_impressions, num_edges, \
#     index_Imps, index_sizeCamps):
#     dictImpPerCamp = {}
#     dictCampPerImp = {}
#     count = 0
#     for campNumber, impsInCamp in enumerate(index_sizeCamps):
#         dictImpPerCamp[campNumber] = index_Imps[count:(count+impsInCamp)]
#         count += impsInCamp

#     ## This is slow, but is sufficient

#     for impType in range(num_impressions):
#         dictCampPerImp[impType] = []

#     for camp in range(numCampaigns):
#         for impType in dictImpPerCamp[camp]:
#             dictCampPerImp[impType].append(camp)

#     return dictImpPerCamp, dictCampPerImp



In [ ]:
# [mat_r_by_Imp, mat_ctr, mat_rctr_by_Imp, ranking, optBidGreedySP, optBidGreedyFP] =\
#     CreateMatR_ctr_Rctr_AndRankings(numCampaigns, num_impressions, num_edges, \
#             index_Imps, index_sizeCamps, vector_r, vector_ctr, \
#             vector_rctr,  UB_bidsPerImp, adverPerImp)